Работа с объектами-контейнерами и настройка spaCy
под свои нужды
========================
05.10.2022

Основные объекты, из которых состоит API spaCy, можно разделить на две категории: контейнеры (например, объекты Token и Doc) и компоненты конвейеров обработки (например, средства частеречной разметки и распознавания именованных сущностей). В этой главе продолжим изучение объектов-контейнеров: благодаря им и их методам можно получать доступ к лингвистическим меткам, которые библиотека spaCy присваивает всем токенам в тексте.

In [59]:
import spacy as sp
nlp = sp.load('ru_core_news_sm')
input_1 = 'Здравствуйте. У меня болит горло..'
input_2 = 'Привет, как дела?'
input_3 = 'Где кошка'
input_4 = 'Хочу купить булочку'


![avatar](./doc.png)

In [60]:
from traceback import print_tb


doc = nlp(input_1)

doc_list = [doc[i] for i in range(len(doc))]   # по индексам
print(doc_list)

# можно обращаться к токенам

print([(token.head.text, token.dep_, token.text) for token in doc])




[Здравствуйте, ., У, меня, болит, горло, ..]
[('Здравствуйте', 'ROOT', 'Здравствуйте'), ('Здравствуйте', 'punct', '.'), ('меня', 'case', 'У'), ('болит', 'obl', 'меня'), ('болит', 'ROOT', 'болит'), ('болит', 'nsubj', 'горло'), ('болит', 'punct', '..')]


Пусть нам нужно найти левосторонний дочерний элемент токена в дереве синтаксических зависимостей предложения. Данная операция
позволяет найти прилагательные (при их наличии) для заданного
существительного. Этоможет понадобиться, если нужно узнать, какие
прилагательные могут модифицировать заданное существительное.

Можно также использовать атрибут Token.rights для получения правосторонних синтаксических дочерних элементов

In [61]:
print(doc[4]) # слово
print([w for w in doc[4].lefts]) # левостороннее дочернее

болит
[меня]


##### Контейнер doc.sents
doc.sents объекта Doc текст можно разделить на
отдельные предложения

In [62]:
print(input_1)
doc = nlp(input_1)

for sent in doc.sents:
    print([sent[i] for i in range(len(sent))])

Здравствуйте. У меня болит горло..
[Здравствуйте, .]
[У, меня, болит, горло, ..]


В то же время можно ссылаться на токены в состоящем из множества
предложений тексте с помощью глобальных индексов уровня документа, как показано вот здесь:

In [63]:
[doc[i] for i in range(len(doc))]

[Здравствуйте, ., У, меня, болит, горло, ..]

In [64]:
doc = nlp(u'Здравствуйте Денис. Я могу предложить вам чай')


for i,sent in enumerate(doc.sents):
    if i==1 and sent[0].pos_== 'PRON':
        print('The second sentence begins with a pronoun.')

The second sentence begins with a pronoun.


In [65]:
# сколько предложений в текстве оканчивается глаголом?
doc = nlp(u'Здравствуйте Денис.я хочу пить. Ты не хочешь. Ну и бог с тобой')

counter = 0
for sent in doc.sents:
    if sent[len(sent)-2].pos_ == 'VERB':
        counter+=1

print('Предлодений с глаглом на конце:', counter)

Предлодений с глаглом на конце: 2


##### Контейнер doc.noun_chunks

С помощью свойства doc.noun_chunks объекта Doc можно пройти по
именным фрагментам. Именной фрагмент (noun chunk) — это фраза,
главнымэлементомкоторой является существительное. (НЕ РАБОТАЕТ)

In [66]:


# сделаем это же вручную
doc = nlp('Собака роет глубокую яму. Рот собаки в грязи')
print([(token.pos_, token.text) for token in doc])


for token in doc:
    if token.pos_=='NOUN':  # ищем существительное
        chunk = ''
        for w in token.children:
            if w.pos_ == 'DET' or w.pos_ == 'ADJ':
                chunk = chunk + w.text + ' '
        chunk = chunk + token.text     
        print(chunk)

[('NOUN', 'Собака'), ('VERB', 'роет'), ('ADJ', 'глубокую'), ('NOUN', 'яму'), ('PUNCT', '.'), ('NUM', 'Рот'), ('NOUN', 'собаки'), ('ADP', 'в'), ('NOUN', 'грязи')]
Собака
глубокую яму
собаки
грязи


Обратите внимание:используемые длямодификациисуществительных
слова (определителии прилагательные)всегдаявляютсялевосторонними
дочерними элементами для существительного. 

In [67]:

for token in doc:
  if token.pos_=='NOUN':  # ищем существительное
        chunk = ''
        for w in token.lefts:
            chunk = chunk + w.text + ' '
        chunk = chunk + token.text     
        print(chunk)

Собака
глубокую яму
Рот собаки
в грязи


#### Объект Span

Объект Span (от англ. span — «интервал») представляет собой часть
объекта Doc.

In [68]:
doc = nlp('Привет, Арбуз будешь?')
doc[2:5] # срезы

Арбуз будешь?

In [69]:
# span.merge()
doc = nlp('Я еду из Ростова в Санк-Петербург на поезде')

[doc[i] for i in range(len(doc))]



[Я, еду, из, Ростова, в, Санк, -, Петербург, на, поезде]

In [70]:

print(5*'-' + 'later merge ' + 5*'-')
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

span = doc[5:8]
print('------- what merge:')
print(span)
print('-------')
lem_id = doc.vocab.strings[span.text]

with doc.retokenize() as retokenizer:
    retokenizer.merge(span)


print(5*'-' + 'after merge: Санкт-Петербург в одну лемму' + 5*'-')
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)


-----later merge -----
Я PRON nsubj еду
еду VERB ROOT еду
из ADP case Ростова
Ростова PROPN obl еду
в ADP case Санк
Санк PROPN obl еду
- PROPN obl еду
Петербург PROPN obl еду
на ADP case поезде
поезде NOUN obl еду
------- what merge:
Санк-Петербург
-------
-----after merge: Санкт-Петербург в одну лемму-----
Я PRON nsubj еду
еду VERB ROOT еду
из ADP case Ростова
Ростова PROPN obl еду
в ADP case Санк-Петербург
Санк-Петербург PROPN obl еду
на ADP case поезде
поезде NOUN obl еду


#### Настройка конвейера обработки текста под свои нужды

Из предыдущих разделов вы узнали, что объекты-контейнеры spaCy
соответствуют различным лингвистическим единицам (например,
целому тексту или отдельному токену), что позволяет извлекать линг-
вистические признаки этих единиц. Теперь же рассмотрим объекты
API spaCy, предназначенные для создания контейнеров и заполнения
их подходящими данными

In [73]:
nlp.pipe_names #Посмотреть доступные для объекта nlp компоненты конвейера можно с помощью команды:

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

#### Отключение компонентов конвейера

spaCy позволяет выбирать загружаемые компоненты конвейера и отключать те, которые не нужны. Это можно сделать при создании объекта nlp, задав параметр disable:

In [76]:
nlp = sp.load('ru_core_news_sm', disable=['parser'])

Данном случае мы создадим конвейер обработки без утилиты разбора зависимостей. При вызове такого экземпляра nlp для конкретного
текста токены в этом тексте не получат метки зависимостей. Вышесказанное наглядно иллюстрирует следующий пример:


In [80]:
doc = nlp(u'Я хочу скушать яблочко')
for token in doc:
    print(token.text, token.pos_, token.dep_)

#Однако метки зависимостей выведены не были.

Я PRON 
хочу VERB 
скушать VERB 
яблочко NOUN 


#### Пошаговая загрузка модели

С помощью метода spacy.load(), загружающего модель, за один раз
можно произвести несколько операций. Например, при выполнении
оператора:
nlp = spacy.load('en')
библиотека spaCy осуществляет такие действия.
1. По названию загружаемой модели определяет, какой экземпляр
класса Language следует инициализировать. В данном примере spaCy
создает экземпляр подкласса English, включающий общий словарь
и прочие языковые данные.
2. Проходит в цикле по названиям компонентов конвейера обработки,
создает соответствующие компоненты и добавляет их в конвейер
обработки.
3. Загружает данные модели с диска, делая их доступными для экземпляра класса Language.

Метод spacy.load() скрывает эти нюансы реализации, в большинстве
случаев экономя ваши силы и время. Но иногда для более тонкой
настройки процесса эти шаги приходится реализовывать явным образом: подобное, например, может понадобиться для подключения
к конвейеру обработки пользовательского компонента для вывода
какой-либо информации об объекте Doc (числа токенов, наличия/отсутствия определенных частей речи и т. д.).

Чем шире возможности настройки, тембольше информации требуется
указать. Чтобы узнать путь к пакету модели, лучше не использовать
сокращенное наименование, а получить настоящее название модели.

In [82]:
print(nlp.meta['lang'] + '_' + nlp.meta['name'])

ru_core_news_sm


In [84]:
print(nlp.meta['lang'] + '_' + nlp.meta['name'] + '-' + nlp.
meta['version'])

ru_core_news_sm-3.4.0


In [85]:
nlp.meta['pipeline'] #список компонентов конвейера, используемых с моделью

['tok2vec', 'morphologizer', 'attribute_ruler', 'lemmatizer', 'ner']

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.tagger.Tagger object at 0x0000020EA25AD820> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.